# Exploring IPV6
<i>Adapted for use with FABRIC from [OSPF](https://www.cs.unc.edu/Research/geni/geniEdu/10-IPv6.html)</i>
    
In this tutorial you will experiment with IPV6.
    
<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-3 at https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/)
    
 In the 1990s, it was realized that the 32 bits allotted for IPV4 addressing would quickly run out.The Internet Engineering Task Force (IETF) designed IPV6 to solve this problem. IPV6 uses 128 bits for addressing allowing for up to 2<sup>128</sup> unique addresses. Transitioning to IPV6 is slow, thus, IPV4 functionality is maintained. To deal with the coexistance you can either dual stack or tunnel. The dual stack method allows for devices to handle both protocols, while tunneling places the IPV6 packet in the data of IPV4 during transit on older protocols.
    
Addresses are written in 8 sets of 4 hexadecimal digits as so:
    <br>`fe80:ab70:0000:0000:0000:0000:0f68:9786`
    
The addresses are abbreviated by not writing leading zeros and cutting out a 4 digit set of 0s and replacing with ::. thus, the above address becomes:
    <br>`fe80:ab70::f68:9786`
    
    Generally the first 3 sets of bits are used for global routing, 1 set for internal routing and 4 sets for node addresses.
    
<img src="./figures/add.png">

## 1. Design the Experiment
In this section, be careful to do the instructions listed with **"Do this"**, as well as running the code blocks.
### 1.1 Reserve Resources

#### Import the Fabric API

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

#### Create slice

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name="IPV6")
    
    #Router
    router = slice.add_node(name="router", site="MAX")
    router.set_capacities(cores=4, ram=16, disk=50)
    router.set_image("default_ubuntu_20")
    rPort1 = router.add_component(model='NIC_Basic', name="rPort1").get_interfaces()[0] 
    rPort2 = router.add_component(model='NIC_Basic', name="rPort2").get_interfaces()[0] 
    
    #Host 1
    host1 = slice.add_node(name="host1", site="MAX")
    host1.set_capacities(cores=4, ram=16, disk=50)
    host1.set_image("default_ubuntu_20")
    h1Port = host1.add_component(model='NIC_Basic', name="h1Port").get_interfaces()[0] 
    
    #Host 2
    host2 = slice.add_node(name="host2", site="MAX")
    host2.set_capacities(cores=4, ram=16, disk=50)
    host2.set_image("default_ubuntu_20")
    h2Port = host2.add_component(model='NIC_Basic', name="h2Port").get_interfaces()[0] 
    
    lan1 = slice.add_l2network(name="Lan1", interfaces=[rPort1, h1Port])
    lan2 = slice.add_l2network(name="Lan2", interfaces=[rPort2, h2Port])
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:    
    host1 = slice.get_node(name="host1") 
    host2 = slice.get_node(name="host2")
    router = slice.get_node(name="router")
    
    subnet1 = IPv4Network("10.1.1.0/24")
    subnet2 = IPv4Network("11.1.1.0/24")
    
    subnet3 = IPv6Network("fe80::/64")
    subnet4 = IPv6Network("fe81::/64")
    
    host1_iface = host1.get_interface(network_name="Lan1")
    host1_iface.ip_addr_add(addr="10.1.1.1", subnet=subnet1)
    host1_iface.ip_addr_add(addr="fe80::1", subnet=subnet3)
    
    router_iface = router.get_interface(network_name="Lan1")
    router_iface.ip_addr_add(addr="10.1.1.2", subnet=subnet1)
    router_iface.ip_addr_add(addr="fe80::2", subnet=subnet3)
    
    router_iface2 = router.get_interface(network_name="Lan2")
    router_iface2.ip_addr_add(addr="11.1.1.2", subnet=subnet2)
    router_iface2.ip_addr_add(addr="fe81::1", subnet=subnet4)
    
    host2_iface2 = host2.get_interface(network_name="Lan2")
    host2_iface2.ip_addr_add(addr="11.1.1.1", subnet=subnet2) 
    host2_iface2.ip_addr_add(addr="fe81::2",subnet=subnet4) 
    
    host1.execute("sudo ip route add 11.1.1.0/24 via 10.1.1.2")
    host1.execute("sudo ip route add fe81::/64 via fe80::2")
    router.execute("sudo sysctl -w net.ipv4.ip_forward=1")
    host2.execute("sudo ip route add 10.1.1.0/24 via 11.1.1.2")
    host2.execute("sudo ip route add fe80::/64/24 via e81::1")
except Exception as e:
    print(f"Exception: {e}")

for node in slice.get_nodes():
    node.execute('sudo apt install net-tools')

## 2. Experiment
### 2.1 Basic Connection
We will be connecting to a node using IPV6.
1. Open a terminal to all three nodes.

2. On `node1` run:
    <br>`ifconfig`
    <br> We can see the addresses of the interfaces, take not of the loopback address (lo). The IPV4 address should be `127.0.0.1`, and the IPV6 address should be `::1`.
    
3. On `node1` ping `node1`:
    <br>Using IPV4: `ping 127.0.0.1 -c 5`
    <br>Using IPV6: `ping6 ::1 -c 5`
4. Ping the multicast address to discover all link local neighbors on `node1` with:
<br>`ping6 -I <link-name> ff02::1 -c 105`
<br>Where `<link-name>` is the name of the interface on a network (ex: ens7). This command will discover all link local neighbors, in this example the output will be from the router node.

5. Ping the router node from `node1`:
<br>`ping6 -I <link-name> <IP6-address> -c 5`
<br>The link used must be specified as in IPV6, link local addressing does not involve routing.
    
6. If you try to ping `node2` from `node1` you will see it fails as you are using link local addressing and have not advertised any additional routes.
### 2.2 Examine Headers
We will examine the difference between IPV4 and IPV6 packets.
1. On `node1` startup tcpdump using:
<br>`sudo tcpdump -v -i <outgoing interface>`
<br> Where the `<outgoing interface>` is the interface we will be grabbing traffic from (found with `ifconfig`).
    
2. On `router`  ping `node1` using IPV4:
<br>`ping 10.1.1.1 -c 5`
<br> Take note of the output on `node1`.
    
3. On `router`  ping `node1` using IPV6:
<br>`ping6 -I <outgoing link name> fe80::1 -c 5`
<br>Where `<outgoing link name>` is the link connected to `node1`.
    
4. Take a look at the differences in packets caught by tcmpdump:
    * IPV6 has a "flow" field, intended to be used for time important transfers
    * IPV6 headers have a standard 40 bytes, if options are needed they are pointed to in the next header field
    
    * Field names have been updated to better reflect their use
    * IPV6 doesn't allow for fragmentation
    
### 2.3 Use a 'Global' Type Address
We will be assigning global type interfaces to the nodes so that they can be pinged without specifying an interface.

1. Choose an address for `node1` (suggested to use the range `2001:0db8::/32` which has been set aside for tutorials and documentation) and run:
<br>`sudo ip -6 addr add <ip6-address>/64 dev <outgoing-link-name>`
<br> Remember that global unicast addresses begin with 001 in the first three bits. Additionally, do not worry about overlap as this address is not facing the internet.

2. Ping the address you just set from `node1` (pinging `node1`):
<br>`ping6 <ip6-address> -c 5`

3. The address isn't advertised yet, verify on `node1` with:
<br>`ip -6 route`
4. Setup route to be advertised, on `node1`:
<br>`sudo ip route add <truncated-ip6-address>/64 dev <outgoing-link-name>`
<br>An example of: `<truncated-ip6-address>` if you chose `2001:db8::1` as your address, then you would use `2001:db8::/64` as your truncated address
5. Run through steps 1-4 on the `router` node using a different ip.
6. Try pinging `node1` from the `router` using the new addresses.

## 3. Cleanup Resources
### 3.1 Delete Slice

In [ ]:
try:
    slice = fablib.get_slice("IPV6")
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")